# 02 – External Data Scraping (Hospitals + Weather)

ใน notebook นี้ เราจะดึง **ข้อมูลภายนอก (external data)** 2 แหล่งหลัก:

1. รายชื่อโรงพยาบาลในกรุงเทพ (Bangkok network hospitals)  
   - ดึงด้วย `requests + BeautifulSoup` → แสดงการทำ web scraping
2. ข้อมูลสภาพอากาศรายชั่วโมงของกรุงเทพ (Hourly weather)  
   - ดึงจาก Open-Meteo API → ใช้เป็นบริบทเวลา (temporal context) สำหรับโมเดล ML

เป้าหมาย:
- เตรียม external data อย่างน้อย 1,000 records ตามข้อกำหนดของ assignment
- เก็บข้อมูลในรูปแบบ CSV/Parquet เพื่อนำไปใช้ใน `03_feature_engineering.ipynb`


In [6]:
import os
import logging
from typing import List, Dict

import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")
EXTERNAL_DIR = os.path.join(DATA_DIR, "external")

os.makedirs(EXTERNAL_DIR, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
)
logger = logging.getLogger("external_scraping")


## Part 1 – Scraping Bangkok Network Hospitals (BeautifulSoup)

แหล่งข้อมูล:  
- ThaiHealth – List of network hospitals in Bangkok area :contentReference[oaicite:1]{index=1}  

ตารางบนเว็บนี้มีคอลัมน์:
- ลำดับ
- ชื่อสถานพยาบาล (ไทย)
- เขต (ไทย)
- โทรศัพท์
- ชื่อสถานพยาบาล (อังกฤษ)
- District (อังกฤษ)
- Telephone (EN)

สิ่งที่เราจะทำ:
1. ใช้ `requests` ดาวน์โหลด HTML
2. ใช้ `BeautifulSoup` หา `<table>` และดึง `<tr><td>` ออกมา
3. แปลงเป็น `pandas.DataFrame`
4. บันทึกเป็น `bangkok_hospitals_network.csv`

**เหตุผลด้าน ML:**  
ภายหลังเราจะ aggregate เป็น feature เช่น  
`num_hospitals_in_district` สำหรับแต่ละเขต → บอกระดับ infrastructure ของพื้นที่นั้น


In [7]:
HOSPITAL_URL = "https://www.thaihealth.co.th/en/list-of-network-hospital-bangkok-area/"

def scrape_bangkok_hospitals(url: str = HOSPITAL_URL) -> pd.DataFrame:
    logger.info("Requesting hospital page: %s", url)
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()
    
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # หา table ตัวหลัก (เว็บนี้มีตารางหลักอันเดียว)
    table = soup.find("table")
    if table is None:
        raise RuntimeError("Cannot find <table> on hospital page.")
    
    rows = table.find_all("tr")
    logger.info("Found %d table rows (including header).", len(rows))
    
    records: List[Dict] = []
    # ข้ามแถว header ตัวแรก
    for tr in rows[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        if not cells:
            continue
        
        # โครงคอลัมน์โดยประมาณ:
        # 0: index
        # 1: hospital_th
        # 2: district_th
        # 3: tel_th
        # 4: hospital_en
        # 5: district_en
        # 6: tel_en
        # ถ้าเว็บเปลี่ยน structure อาจต้องปรับ mapping ตรงนี้
        if len(cells) < 4:
            continue
        
        # ป้องกัน index error แบบหยาบ ๆ
        # ใช้ min(len, expected_index) ตัดให้พอ
        def safe(i): 
            return cells[i] if i < len(cells) else ""
        
        record = {
            "hospital_th": safe(1),
            "district_th": safe(2),
            "tel_th": safe(3),
            "hospital_en": safe(4),
            "district_en": safe(5),
            "tel_en": safe(6),
        }
        records.append(record)
    
    df = pd.DataFrame(records)
    logger.info("Parsed %d hospital rows.", len(df))
    return df

df_hospitals = scrape_bangkok_hospitals()
df_hospitals.head()


2025-11-24 14:09:18,840 | INFO | external_scraping | Requesting hospital page: https://www.thaihealth.co.th/en/list-of-network-hospital-bangkok-area/
2025-11-24 14:09:20,138 | INFO | external_scraping | Found 110 table rows (including header).
2025-11-24 14:09:20,140 | INFO | external_scraping | Parsed 109 hospital rows.


,hospital_th,district_th,tel_th,hospital_en,district_en,tel_en
0,กรุงเทพ,ห้วยขวาง,02-3103000,Bangkok Hospital,Huai Khwang,02-3103000
1,กรุงเทพคริสเตียน,บางรัก,02-6259000,The Bangkok Christian,Bang Rak,02-6259000
2,กล้วยน้ำไท,คลองเตย,02-7692000,Kluaynamthai,Khlong Toei,02-7692000
3,กล้วยน้ำไท 2,บางนา,02-3994259-63,Kluaynamthai 2,Bang Na,02-3994259-63
4,เกษมราษฎร์ รามคำแหง,สะพานสูง,02-339-0000,Kasemrad Ramkhamhaeng,Saphan Sung,02-339-000


In [8]:
# clean เบื้องต้น
df_hospitals["district_th"] = df_hospitals["district_th"].str.strip()
df_hospitals["district_en"] = df_hospitals["district_en"].str.strip()

hospital_path = os.path.join(EXTERNAL_DIR, "bangkok_hospitals_network.csv")
df_hospitals.to_csv(hospital_path, index=False, encoding="utf-8-sig")

logger.info("Saved hospital network data to: %s", hospital_path)
logger.info("Total hospital rows: %d", len(df_hospitals))


2025-11-24 14:09:20,153 | INFO | external_scraping | Saved hospital network data to: /home/frostnzx/uniworks/datasci/final_project/data/external/bangkok_hospitals_network.csv
2025-11-24 14:09:20,153 | INFO | external_scraping | Total hospital rows: 109


## Part 2 – Download Hourly Weather for Bangkok (API)

แหล่งข้อมูล:  
- Open-Meteo API (ฟรี ไม่ต้องใช้ API key)

เราอยากได้:
- ปริมาณฝนต่อชั่วโมง (`precipitation`)
- อุณหภูมิ (`temperature_2m`)
- ความเร็วลม (`wind_speed_10m`)

ช่วงเวลา:
- ตัวอย่าง: ดึงข้อมูล 1 ปี (เช่น ปี 2023) → 8,000+ rows  
  ซึ่งเพียงพอสำหรับ requirement ≥ 1,000 records

**เหตุผลด้าน ML:**

ฟีเจอร์จากสภาพอากาศที่อาจใช้:
- `is_rainy_hour` (ฝน > threshold)
- `rain_mm_last_3h`
- `temp_2m`
- `wind_speed_10m`

ฝนตก / อากาศแย่ มักทำให้:
- งานภาคสนามช้ากว่าเดิม
- ปริมาณ ticket ที่เกี่ยวกับน้ำท่วม/ถนนเพิ่มขึ้น
- โอกาสการแก้ปัญหา “late” สูงขึ้น


In [9]:
from datetime import date

# พิกัดกรุงเทพคร่าว ๆ
BANGKOK_LAT = 13.75
BANGKOK_LON = 100.50

# กำหนดช่วงเวลา (ต้องครอบคลุมช่วงเวลาของ Traffy)
start_date = "2023-01-01"
end_date = "2023-12-31"

WEATHER_URL = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": BANGKOK_LAT,
    "longitude": BANGKOK_LON,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": ["temperature_2m", "precipitation", "wind_speed_10m"],
    "timezone": "Asia/Bangkok",
}

logger.info("Requesting hourly weather from Open-Meteo...")
resp = requests.get(WEATHER_URL, params=params, timeout=30)
resp.raise_for_status()
weather_json = resp.json()

# แปลง JSON → DataFrame
hourly = weather_json["hourly"]
df_weather = pd.DataFrame(hourly)

logger.info("Weather rows: %d", len(df_weather))
df_weather.head()


2025-11-24 14:09:20,167 | INFO | external_scraping | Requesting hourly weather from Open-Meteo...
2025-11-24 14:09:21,929 | INFO | external_scraping | Weather rows: 8760


,time,temperature_2m,precipitation,wind_speed_10m
0,2023-01-01T00:00,23.3,0.0,9.8
1,2023-01-01T01:00,22.4,0.0,9.7
2,2023-01-01T02:00,21.6,0.0,9.2
3,2023-01-01T03:00,20.9,0.0,8.5
4,2023-01-01T04:00,20.3,0.0,7.4


In [10]:
# เปลี่ยนชื่อ column เวลา
df_weather.rename(columns={"time": "datetime"}, inplace=True)

weather_path = os.path.join(EXTERNAL_DIR, "bangkok_hourly_weather_2023.csv")
df_weather.to_csv(weather_path, index=False, encoding="utf-8-sig")

logger.info("Saved hourly weather data to: %s", weather_path)


2025-11-24 14:09:21,958 | INFO | external_scraping | Saved hourly weather data to: /home/frostnzx/uniworks/datasci/final_project/data/external/bangkok_hourly_weather_2023.csv


# Summary – External Data Ready

ในตอนนี้ เรามี external data 2 ชุด:

1. `bangkok_hospitals_network.csv`
   - แหล่งที่มา: ThaiHealth (web scraping ด้วย BeautifulSoup)
   - ข้อมูล: ชื่อโรงพยาบาล, เขต, เบอร์โทร (TH/EN)
   - ใช้ต่อ: สร้างฟีเจอร์ระดับเขต เช่น `num_hospitals_in_district`

2. `bangkok_hourly_weather_2023.csv`
   - แหล่งที่มา: Open-Meteo API (hourly weather)
   - ข้อมูล: เวลา, อุณหภูมิ, ปริมาณฝน, ความเร็วลม
   - ใช้ต่อ: join กับ Traffy ตามเวลา → ฟีเจอร์ด้านสภาพอากาศ

สอง dataset นี้:
- รวมกันมี records > 1,000 แถว (ผ่าน requirement ของ assignment)
- มีความเชื่อมโยงกับปัญหา "late vs not-late" (ด้าน infrastructure + สภาพอากาศ)
- แสดงให้เห็นทั้ง **web scraping (BeautifulSoup)** และ **API integration**

ขั้นตอนถัดไป (ใน `03_feature_engineering.ipynb`):

1. โหลด `traffy_clean.parquet` จาก `_prep`
2. Map เขตของ Traffy ↔ เขตของ `bangkok_hospitals_network`
3. Aggregate จำนวนโรงพยาบาลต่อเขต
4. Join ข้อมูลสภาพอากาศตาม `timestamp → closest datetime`
5. สร้างฟีเจอร์ใหม่ เช่น `num_hospitals_in_district`, `is_rainy_hour`, `rain_mm_last_3h`
